# Models

## distilbert-base-uncased-finetuned-sst-2-english

distilbert-base-uncased-finetuned-sst-2-english
Binary sentiment (POSITIVE / NEGATIVE)
https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english

## SamLowe/roberta-base-go_emotions	

SamLowe/roberta-base-go_emotions	
28 fine-grained emotions (admiration → worry)
https://huggingface.co/SamLowe/roberta-base-go_emotions

## unitary/unbiased-toxic-roberta	

unitary/unbiased-toxic-roberta	

Toxicity & six sub-types (toxic, severe_toxic, obscene, etc.)


https://huggingface.co/unitary/unbiased-toxic-roberta

## Hate-speech-CNERG/dehatebert-mono-english

Hate-speech-CNERG/dehatebert-mono-english
Hate / non-hate
https://huggingface.co/Hate-speech-CNERG/dehatebert-mono-english

## classla/multilingual-IPTC-news-topic-classifier

classla/multilingual-IPTC-news-topic-classifier

205 IPTC NewsCodes topics (e.g., crime, culture, health)

https://huggingface.co/classla/multilingual-IPTC-news-topic-classifier

# Prerequisites

In [ ]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

In [ ]:
# pip install transformers datasets evaluate bitsandbytes accelerate peft

Check that CUDA is enabled

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available in PyTorch: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Chosen model

distilbert-base-uncased-finetuned-sst-2-english

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

ds = load_dataset("go_emotions", "simplified")

tok_r = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
mdl_r = AutoModelForSequenceClassification.from_pretrained(
            "SamLowe/roberta-base-go_emotions").to("cuda")

clf = pipeline("text-classification",
               model=mdl_r, tokenizer=tok_r,
               device=0, batch_size=32, top_k=None)

def predict_split(split):
    out = clf(split["text"])
    return [{d["label"]: d["score"] for d in row} for row in out]

valid_logits  = predict_split(ds["validation"])
test_logits   = predict_split(ds["test"])
ds["validation"] = ds["validation"].add_column("roberta_logits", valid_logits)
ds["test"]       = ds["test"].add_column("roberta_logits",  test_logits)


In [ ]:
print(ds['train'][0])

# Creating the train data set

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from tqdm import tqdm


In [ ]:
emotion_labels = ds["validation"].features["labels"].feature.names

In [ ]:
def create_emotion_probabilities_df(dataset_split):
    texts = []
    emotion_probs = {emotion: [] for emotion in emotion_labels}
    
    for text, roberta_logits in zip(dataset_split["text"], dataset_split["roberta_logits"]):
        texts.append(text)
        
        for emotion in emotion_labels:
            prob = roberta_logits.get(emotion, 0.0)
            emotion_probs[emotion].append(prob)
    
    df_data = {'input_text': texts}
    df_data.update(emotion_probs)
    
    df = pd.DataFrame(df_data)
    return df

def predict_with_dataset_optimized(dataset_split, batch_size=128):
    texts = dataset_split["text"]
    temp_dataset = Dataset.from_dict({"text": texts})
    
    def predict_batch(batch):
        try:
            outputs = clf(batch["text"], truncation=True, max_length=512)
            predictions = [{d["label"]: d["score"] for d in row} for row in outputs]
            return {"roberta_logits": predictions}
        except Exception as e:
            empty_preds = [{label: 0.0 for label in emotion_labels} for _ in range(len(batch["text"]))]
            return {"roberta_logits": empty_preds}
    
    result_dataset = temp_dataset.map(
        predict_batch,
        batched=True,
        batch_size=batch_size,
        remove_columns=["text"]
    )
    
    predictions = result_dataset["roberta_logits"]
    return predictions

def predict_with_dataset_progress(dataset_split, batch_size=128):
    texts = dataset_split["text"]
    total_samples = len(texts)
    chunk_size = 5000
    all_predictions = []
    
    for start_idx in tqdm(range(0, total_samples, chunk_size)):
        end_idx = min(start_idx + chunk_size, total_samples)
        chunk_texts = texts[start_idx:end_idx]
        
        chunk_dataset = Dataset.from_dict({"text": chunk_texts})
        
        def predict_batch(batch):
            outputs = clf(batch["text"], truncation=True, max_length=512)
            return {"roberta_logits": [{d["label"]: d["score"] for d in row} for row in outputs]}
        
        chunk_result = chunk_dataset.map(
            predict_batch,
            batched=True,
            batch_size=batch_size,
            remove_columns=["text"]
        )
        
        all_predictions.extend(chunk_result["roberta_logits"])
    
    return all_predictions

if 'roberta_logits' not in ds['train'].column_names:
    try:
        train_logits = predict_with_dataset_optimized(ds["train"], batch_size=128)
    except Exception as e:
        train_logits = predict_with_dataset_progress(ds["train"], batch_size=64)
    
    ds["train"] = ds["train"].add_column("roberta_logits", train_logits)

train_df = create_emotion_probabilities_df(ds["train"])
validation_df = create_emotion_probabilities_df(ds["validation"])
test_df = create_emotion_probabilities_df(ds["test"])

In [ ]:
train_df = create_emotion_probabilities_df(ds["train"])

In [ ]:
train_df

In [ ]:
train_df.columns

In [ ]:
emotion_cols = train_df.columns.drop("input_text")
emotion_values = train_df[emotion_cols].values
emotion_values

In [ ]:

top_3_indices = np.argpartition(emotion_values, -3, axis=1)[:, -3:]
top_3_sorted = np.take_along_axis(top_3_indices,
                                  np.argsort(np.take_along_axis(emotion_values, top_3_indices, axis=1), axis=1)[:, ::-1],
                                  axis=1)
train_df['emotion_1'] = [emotion_cols[idx] for idx in top_3_sorted[:, 0]]
train_df['emotion_2'] = [emotion_cols[idx] for idx in top_3_sorted[:, 1]]
train_df['emotion_3'] = [emotion_cols[idx] for idx in top_3_sorted[:, 2]]

train_df['emotion_1_value'] = emotion_values[np.arange(len(emotion_values)), top_3_sorted[:, 0]]
train_df['emotion_2_value'] = emotion_values[np.arange(len(emotion_values)), top_3_sorted[:, 1]]
train_df['emotion_3_value'] = emotion_values[np.arange(len(emotion_values)), top_3_sorted[:, 2]]

In [ ]:
train_df

## Clearly seperated

In [ ]:
train_df_clearly_seperated = train_df.copy()

mask = (train_df_clearly_seperated['emotion_1_value'] > 0.6) & \
       (train_df_clearly_seperated['emotion_1_value'] >= 2 * train_df_clearly_seperated['emotion_2_value'])

train_df_clearly_seperated = train_df_clearly_seperated[mask]

print(f"Original dataset size: {len(train_df)}")
print(f"Filtered dataset size: {len(train_df_clearly_seperated)}")
print(f"Percentage retained: {len(train_df_clearly_seperated) / len(train_df) * 100:.2f}%")
train_df_clearly_seperated

In [ ]:
train_df.columns

In [ ]:
input_output_df = train_df_clearly_seperated.copy()[["input_text", "emotion_1"]].rename(columns={"input_text": "input", "emotion_1": "target"}).reset_index(drop=True)
input_output_df

In [ ]:
input_output_df.to_csv("clearly_seperated.csv")

## All

In [ ]:
input_output_df = train_df.copy()[["input_text", "emotion_1"]].rename(columns={"input_text": "input", "emotion_1": "target"}).reset_index(drop=True)
input_output_df

# Prompt engineering

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=quantization_config
)

In [ ]:
def get_model_prediction(input_text, prompt_template=None, max_new_tokens=10, do_sample=False, temperature=0.7):
    if prompt_template is None:
        prompt = f"What single concept best explains the output? Input: {input_text}"
    else:
        prompt = prompt_template.format(input_text=input_text)
   
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
   
    generation_kwargs = {
        "max_new_tokens": max_new_tokens,
        "do_sample": do_sample,
        "pad_token_id": tokenizer.eos_token_id
    }
    
    if do_sample:
        generation_kwargs["temperature"] = temperature
   
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **generation_kwargs
        )
   
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated_text.strip()

In [ ]:
first_10 = input_output_df.head(10).copy()
prompts = {
    "concept": "What single concept best explains the output? Input: {input_text}",
    "emotion": "What emotion does this text convey? Text: {input_text}\nEmotion:",
    "topic": "Identify the main topic of this text in one word: {input_text}\nTopic:",
    "summary": "Summarize this in one word: {input_text}\nWord:"
}

for prompt_name, prompt_template in prompts.items():
    first_10[f'pred_{prompt_name}'] = first_10['input'].apply(
        lambda x: get_model_prediction(
            x, 
            prompt_template=prompt_template,
            max_new_tokens=10,
            do_sample=prompt_name != "concept",
            temperature=0.1
        )
    )

first_10

In [ ]:
grouped_samples = input_output_df.groupby('target').head(10).copy()

In [ ]:
concept_prompt = "What single concept best explains the output? Input: {input_text}"

grouped_samples['concept_prediction'] = grouped_samples['input'].apply(
    lambda x: get_model_prediction(
        x,
        prompt_template=concept_prompt,
        max_new_tokens=10,
        do_sample=False
    )
)

result_df = pd.DataFrame({
    'prompt': grouped_samples['input'],
    'output': grouped_samples['concept_prediction'],
    'real_output': grouped_samples['target']
})

result_df = result_df.reset_index(drop=True)

result_df

In [ ]:
concept_prompt = "What single emotion best explains the what the author is feeling? Input: {input_text}"

grouped_samples['concept_prediction'] = grouped_samples['input'].apply(
    lambda x: get_model_prediction(
        x,
        prompt_template=concept_prompt,
        max_new_tokens=10,
        do_sample=False
    )
)

result_df = pd.DataFrame({
    'prompt': grouped_samples['input'],
    'output': grouped_samples['concept_prediction'],
    'real_output': grouped_samples['target']
})

result_df = result_df.reset_index(drop=True)

result_df

In [ ]:
grouped_samples = input_output_df.groupby('target').head(10).copy()

concept_prompt = f"What single emotion out of the following: {', '.join(emotion_cols)} best explains what the author is feeling? Input: {{input_text}}"

grouped_samples['concept_prediction'] = grouped_samples['input'].apply(
    lambda x: get_model_prediction(
        x,
        prompt_template=concept_prompt,
        max_new_tokens=10,
        do_sample=False
    )
)

result_df = pd.DataFrame({
    'prompt': grouped_samples['input'],
    'output': grouped_samples['concept_prediction'],
    'real_output': grouped_samples['target']
})

result_df = result_df.reset_index(drop=True)
result_df

In [ ]:
def get_model_prediction(input_text):
    prompt = f"What single concept best explains the output? Input: {input_text}"
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated_text.strip()

first_10 = input_output_df.head(10).copy()
first_10['model_prediction'] = first_10['input'].apply(get_model_prediction)
first_10[['input', 'target', 'model_prediction']]

In [ ]:
grouped_samples = input_output_df.groupby('target').head(10).copy()

concept_prompt = f"What single emotion out of the following: {', '.join(emotion_cols)} best explains what the author is feeling? Input: {{input_text}} Finish the sentence: Output:The author is feeling: "

grouped_samples['concept_prediction'] = grouped_samples['input'].apply(
    lambda x: get_model_prediction(
        x,
        prompt_template=concept_prompt,
        max_new_tokens=10,
        do_sample=False
    )
)

result_df = pd.DataFrame({
    'prompt': grouped_samples['input'],
    'output': grouped_samples['concept_prediction'],
    'real_output': grouped_samples['target']
})

result_df = result_df.reset_index(drop=True)
result_df

# Evaluation

In [ ]:
def check_prediction(row):
    prediction = row['output'].lower()
    target = row['real_output'].lower()
    if target in prediction:
        return 1
    
    for emotion in emotion_cols:
        if emotion.lower() in prediction and emotion.lower() != target:
            return -1
    return 0

result_df['prediction_true'] = result_df.apply(check_prediction, axis=1)
result_df

In [ ]:
result_df[result_df["prediction_true"] != 0]

In [ ]:
count_of_correct = (result_df[result_df["prediction_true"] != 0]["prediction_true"]==1).sum()
count_of_incorrect = (result_df[result_df["prediction_true"] != 0]["prediction_true"]==-1).sum()
total_count = result_df.shape[0]
print(f"Correct predictions: {count_of_correct} out of {total_count} which is {count_of_correct/total_count*100}%")
print(f"Incorrect predictions: {count_of_incorrect} out of {total_count} which is {count_of_incorrect/total_count*100}%")

Dużo lepiej niż losowe :)

# Input-output function

In [ ]:
def load_llm_model():
    model_name = "microsoft/Phi-3-mini-4k-instruct"
    
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        quantization_config=quantization_config
    )
    
    return tokenizer, model

def get_llm_emotion_prediction(text, tokenizer, model, emotion_list):
    emotions_str = ", ".join(emotion_list)
    prompt = f"What emotion from this list: [{emotions_str}] best describes this text: '{text}' Answer with just the emotion name:"
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    response = generated_text.strip().lower()
    for emotion in emotion_list:
        if emotion.lower() in response:
            return emotion.lower()
    words = response.split()
    return words[0] if words else "unknown"

In [ ]:
test_df = pd.DataFrame({
    'input': ['Life is amazing'],
    'target': ['happiness']
})

test_df

In [ ]:
tokenizer, model = load_llm_model()

In [ ]:
emotion_list = list(emotion_cols)

text = test_df['input'][0]
bert_prediction = test_df['target'][0]

print(f"Input text: '{text}'")
print(f"BERT prediction: '{bert_prediction}'")

llm_prediction = get_llm_emotion_prediction(text, tokenizer, model, emotion_list)

print(f"LLM prediction: '{llm_prediction}'")
print()

In [ ]:
emotion_list = ['happiness', 'sadness', 'anger', 'fear', 'surprise', 'disgust', 'joy', 'love', 'excitement']
text = test_df['input'][0]
bert_prediction = test_df['target'][0]

print(f"Input text: '{text}'")
print(f"BERT prediction: '{bert_prediction}'")

llm_prediction = get_llm_emotion_prediction(text, tokenizer, model, emotion_list)

print(f"LLM prediction: '{llm_prediction}'")
print()

lepiej działa dla małej listy

# Using the implemented classes

## HFM

In [ ]:
from huggingface_model import HuggingFaceModel

In [ ]:
HGF = HuggingFaceModel()

In [ ]:
HGF.predict("I'm happy")

In [ ]:
HGF.predict("I'm sad")

## LLM

In [1]:
import pandas as pd
import sys
import os
current_dir = os.getcwd()
if current_dir.endswith('basic'):
    project_root = os.path.dirname(current_dir)
else:
    project_root = current_dir
project_root
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [2]:
from llm_model import LLMModel

c:\nextcloud\Studia - PW\semestr 6\warsztaty_badawcze\llm-workshop\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# llm_model = LLMModel(model_name="microsoft/DialoGPT-large")
llm_model = LLMModel(model_name="facebook/opt-2.7b")

Loading model: facebook/opt-2.7b
Model loaded successfully on cuda


In [4]:
train_data = pd.read_csv("clearly_seperated.csv").rename(columns={"target":"output"}).head(20)
x_train = train_data["input"].tolist()
y_train = train_data["output"].tolist()
training_pairs = [(x, y) for x, y in zip(x_train, y_train) if pd.notna(x) and pd.notna(y)]
x_train_clean = [pair[0] for pair in training_pairs]
y_train_clean = [pair[1] for pair in training_pairs]

print(f"Training examples: {len(x_train_clean)}")
print(f"Sample training data:")
for i in range(min(3, len(x_train_clean))):
    print(f"  Input: '{x_train_clean[i]}' -> Output: '{y_train_clean[i]}'")

Training examples: 20
Sample training data:
  Input: 'Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead' -> Output: 'neutral'
  Input: 'WHY THE FUCK IS BAYLESS ISOING' -> Output: 'anger'
  Input: 'Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝' -> Output: 'gratitude'


In [5]:
llm_model.train(x_train_clean, y_train_clean)

Concept prediction prompt:
In 2 words guess, what task is the model doing, the format is x_test -> y_test:
Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead -> neutral
WHY THE FUCK IS BAYLESS ISOING -> anger
Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝 -> gratitude
We need more boards and to create a bit more space for [NAME]. Then we’ll be good. -> desire
It might be linked to the trust factor of your friend. -> neutral
Aww... she'll probably come around eventually, I'm sure she was just jealous of [NAME]... I mean, what woman wouldn't be! lol  -> amusement
Hello everyone. Im from Toronto as well. Can call and visit in personal if needed. -> neutral
R/sleeptrain Might be time for some sleep training. Take a look and try to feel out what's right for your family. -> caring
Thank you friend -> gratitude
Fucking coward. -> anger
that is what retardation l

In [6]:
llm_model.predict_concept(x_train, y_train)

Concept prediction prompt:
In 2 words guess, what task is the model doing, the format is x_test -> y_test:
Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead -> neutral
WHY THE FUCK IS BAYLESS ISOING -> anger
Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝 -> gratitude
We need more boards and to create a bit more space for [NAME]. Then we’ll be good. -> desire
It might be linked to the trust factor of your friend. -> neutral
Aww... she'll probably come around eventually, I'm sure she was just jealous of [NAME]... I mean, what woman wouldn't be! lol  -> amusement
Hello everyone. Im from Toronto as well. Can call and visit in personal if needed. -> neutral
R/sleeptrain Might be time for some sleep training. Take a look and try to feel out what's right for your family. -> caring
Thank you friend -> gratitude
Fucking coward. -> anger
that is what retardation l

'philosoph, as a little boy who will not too much more than anything that we are all the first of the most recent history of course, we have been made clear for the same as part in the only one of antonally, and achina, it is a single, my newspinning and it”'

In [7]:
llm_model.predict("I'm happy")

Prediction prompt for 'I'm happy':
You are a classificator
Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead -> neutral
WHY THE FUCK IS BAYLESS ISOING -> anger
Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝 -> gratitude
We need more boards and to create a bit more space for [NAME]. Then we’ll be good. -> desire
It might be linked to the trust factor of your friend. -> neutral
Aww... she'll probably come around eventually, I'm sure she was just jealous of [NAME]... I mean, what woman wouldn't be! lol  -> amusement
Hello everyone. Im from Toronto as well. Can call and visit in personal if needed. -> neutral
R/sleeptrain Might be time for some sleep training. Take a look and try to feel out what's right for your family. -> caring
Thank you friend -> gratitude
Fucking coward. -> anger
that is what retardation looks like -> neutral
Maybe that’s what happened 

['ACHEscape.']

In [8]:
llm_model.predict("I'm sad")

Prediction prompt for 'I'm sad':
You are a classificator
Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead -> neutral
WHY THE FUCK IS BAYLESS ISOING -> anger
Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝 -> gratitude
We need more boards and to create a bit more space for [NAME]. Then we’ll be good. -> desire
It might be linked to the trust factor of your friend. -> neutral
Aww... she'll probably come around eventually, I'm sure she was just jealous of [NAME]... I mean, what woman wouldn't be! lol  -> amusement
Hello everyone. Im from Toronto as well. Can call and visit in personal if needed. -> neutral
R/sleeptrain Might be time for some sleep training. Take a look and try to feel out what's right for your family. -> caring
Thank you friend -> gratitude
Fucking coward. -> anger
that is what retardation looks like -> neutral
Maybe that’s what happened to

['| This is the sunbathecatast. I※s, but that the same as a few months ago, you do not to make you can be a person who is a single-upwardenjoyable and ready to apropermitters are not quite honestly, the world‐toughness. This will not only when we are very much of the case. The majority of this is not long term for each daydreaming, then again, or not a']

In [9]:
train_data # musi miec input i output

,Unnamed: 0,input,output
0,0,"Now if he does off himself, everyone will thin...",neutral
1,1,WHY THE FUCK IS BAYLESS ISOING,anger
2,2,Yes I heard abt the f bombs! That has to be wh...,gratitude
3,3,We need more boards and to create a bit more s...,desire
4,4,It might be linked to the trust factor of your...,neutral
5,5,"Aww... she'll probably come around eventually,...",amusement
6,6,Hello everyone. Im from Toronto as well. Can c...,neutral
7,7,R/sleeptrain Might be time for some sleep trai...,caring
8,8,Thank you friend,gratitude
9,9,Fucking coward.,anger


In [ ]:
test_size = min(3, len(x_train_clean))
x_test = x_train_clean[:test_size]
y_test = y_train_clean[:test_size]

In [12]:
concept = llm_model.predict_concept(x_train, y_train)
print(f"Detected task: {concept}")

Concept prediction prompt:
In 2 words guess, what task is the model doing, the format is x_test -> y_test:
Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead -> neutral
WHY THE FUCK IS BAYLESS ISOING -> anger
Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝 -> gratitude
We need more boards and to create a bit more space for [NAME]. Then we’ll be good. -> desire
It might be linked to the trust factor of your friend. -> neutral
Aww... she'll probably come around eventually, I'm sure she was just jealous of [NAME]... I mean, what woman wouldn't be! lol  -> amusement
Hello everyone. Im from Toronto as well. Can call and visit in personal if needed. -> neutral
R/sleeptrain Might be time for some sleep training. Take a look and try to feel out what's right for your family. -> caring
Thank you friend -> gratitude
Fucking coward. -> anger
that is what retardation l

In [ ]:
from llm_model import LLMModel
LLM_model = LLMModel()
LLM_model.demo_workflow()

In [ ]:
if len(x_train_clean) > 3:
    new_inputs = x_train_clean[-2:]  # Last 2 examples as test
    predictions = llm_model.predict(new_inputs)
    
    print("Prediction results:")
    for inp, pred in zip(new_inputs, predictions):
        print(f"  Input: '{inp}' -> Prediction: '{pred}'")
else:
    print("Not enough data for separate prediction examples")